# SBP-SAT methods

Вызываются из в CiclicScheme1TransportEquation

In [1]:
#-INCLUDE-MODULS-AND-LIBRARYS-
%matplotlib notebook

import numpy              as np
import scipy
import matplotlib.pyplot  as plt
import glob
import moviepy.editor     as mpy

from scipy                import sparse
from scipy.sparse         import linalg
from numpy                import pi, sin, cos, ma
from pylab                import *
from mpl_toolkits.mplot3d import Axes3D
from matplotlib           import cm
from matplotlib.ticker    import LinearLocator, FormatStrFormatter
from matplotlib           import animation

-----
## 1. SBP space derivatives

In [2]:
#-SBP-OPERATORS-FOR-FIRST-SPACE-DERIVATIVES-

def H21(f,u,h): #Inner product SBP-2,1
    
    A     = np.zeros(f.size)+1
    A[ 0] = 1/2
    A[-1] = 1/2
    data  = np.array([A])
    diags = np.array([0])
    H1     = sparse.spdiags(data, diags, f.size, f.size).toarray()
    
    return H1

def H42(f,u,h): #Inner product SBP-4,2 
    
    A     = np.zeros(f.size)+1
    
    A[ 0] = 17/48
    A[ 1] = 59/48
    A[ 2] = 43/48
    A[ 3] = 49/48

    A[-1] = A[0]
    A[-2] = A[1]
    A[-3] = A[2]
    A[-4] = A[3]
    
    data  = np.array([A])
    diags = np.array([0])
    H_42  = sparse.spdiags(data, diags, f.size, f.size).toarray()
    
    return H_42


def Q21(f,u,h): #Pre-difference scheme SBP-2,1
    
    A     = np.zeros(f.size)+1
    B     = np.zeros(f.size)-1
    
    data  = np.array([u/(2*h)*B, u/(2*h)*A])
    diags = np.array([-1, 1])
    Q_21  = sparse.spdiags(data, diags, f.size, f.size).toarray()
    
    Q_21[ 0] = zeros(f.size)
    Q_21[-1] = zeros(f.size)
    
    Q_21[ 0, 0] = -1/(2*h)
    Q_21[ 0, 1] =  1/(2*h)
    Q_21[-1,-1] =  1/(2*h)
    Q_21[-1,-2] = -1/(2*h)

    return Q_21

def Q42(f,u,h): #Pre-difference scheme SBP-4,2
    
    A     = np.zeros(f.size)+1
    B     = np.zeros(f.size)-1
    
    data  = np.array([u/(12*h)*A, 8*u/(12*h)*B, 8*u/(12*h)*A, u/(12*h)*B])
    diags = np.array([-2, -1, 1, 2])
    Q_42  = sparse.spdiags(data, diags, f.size, f.size).toarray()
    
    Q_42[ 0] = zeros(f.size)
    Q_42[ 1] = zeros(f.size)

    Q_42[0,0] = -1/(2*h)
    Q_42[0,1] =  59/(96*h)
    Q_42[0,2] = -1/(12*h)
    Q_42[0,3] = -1/(32*h)
    Q_42[1,2] =  59/(96*h)
    Q_42[1,3] =  0
    Q_42[2,3] =  59/(96*h)
    
    Q_42[1,0] = -Q_42[0,1]
    Q_42[2,0] = -Q_42[0,2]
    Q_42[3,0] = -Q_42[0,3]
    Q_42[2,1] = -Q_42[1,2]
    Q_42[3,1] = -Q_42[1,3]
    Q_42[3,2] = -Q_42[2,3]
    
    Q_42[-1,-1] = -Q_42[0,0]
    Q_42[-1,-2] = -Q_42[0,1]
    Q_42[-1,-3] = -Q_42[0,2]
    Q_42[-1,-4] = -Q_42[0,3]
    Q_42[-2,-3] = -Q_42[1,2]
    Q_42[-2,-4] = -Q_42[1,3]
    Q_42[-3,-4] = -Q_42[2,3]
    
    Q_42[-2,-1] = -Q_42[-1,-2]
    Q_42[-3,-1] = -Q_42[-1,-3]
    Q_42[-4,-1] = -Q_42[-1,-4]
    Q_42[-3,-2] = -Q_42[-2,-3]
    Q_42[-4,-2] = -Q_42[-2,-4]
    Q_42[-4,-3] = -Q_42[-3,-4]

    return Q_42

    
def P(f,u,h): #Projection method (Half-sum at the ends)
    
    A     = np.zeros(f.size)+1
    data  = np.array([A])
    diags = np.array([0])
    P     = sparse.spdiags(data, diags, f.size, f.size).toarray()
    
    P[ 0, 0] = 1/2
    P[ 0,-1] = 1/2
    P[-1,-1] = 1/2
    P[-1, 0] = 1/2
    
    return P

## 2. SAT

In [3]:
#SAT-TERMS-DEFINITION-

def SAT0(f, u, h):
    e0    = np.zeros(f.size)
    e0[0] = 1
    SAT_0 = 1/2*(f[0]-f[-1])*e0
    return SAT_0

def SATN(f, u, h):
    eN     = np.zeros(f.size)
    eN[-1] = 1
    SAT_N  = 1/2*(f[0]-f[-1])*eN
    return SAT_N